# Set Config and Complie

In [ ]:
MODEL_DIR = './ML_CMSL1CaloTrigger/saved_models/qmodel_oct24'

In [ ]:
import numpy as np
import h5py
import tensorflow as tf
import hls4ml

from qkeras import *
from qkeras.utils import load_qmodel

In [ ]:
model = load_qmodel(MODEL_DIR);
model.summary()

In [ ]:
hls4ml.model.optimizer.get_optimizer(
    'output_rounding_saturation_mode'
).configure(
    layers=['relu1', 'relu2'],
    rounding_mode='AP_RND',
    saturation_mode='AP_SAT',
    saturation_bits='AP_SAT'
)
hls4ml.model.optimizer.get_optimizer(
    'eliminate_linear_activation'
)

hls_config = hls4ml.utils.config_from_keras_model(
    model,
    granularity='name'
)

hls_config['Model']['ReuseFactor'] = 4
hls_config['Model']['Strategy'] = 'Resource'
hls_config['Model']['ClockPeriod']  = 6.25
hls_config['Model']['Precision'] = 'ap_fixed<16, 6>'
# hls_config['Model']['Trace']  = True

for layer in hls_config['LayerName'].keys():
    hls_config['LayerName'][layer]['Strategy'] = 'Resource'
    hls_config['LayerName'][layer]['ReuseFactor'] = 4
    # hls_config['LayerName'][layer]['Trace'] = True

hls_config['LayerName']['In']['Precision']['accum'] = 'ap_uint<10>'
hls_config['LayerName']['In']['Precision']['result'] = 'ap_uint<10>'

hls_config['LayerName']['conv']['Precision']['accum'] = 'ap_fixed<20, 8>'
hls_config['LayerName']['conv']['Precision']['result'] = 'ap_fixed<15, 8>'
hls_config['LayerName']['conv_linear']['Precision'] = 'ap_fixed<15, 8>'
hls_config['LayerName']['conv']['Strategy'] = 'Resource'
hls_config['LayerName']['conv']['ReuseFactor'] = 1
hls_config['LayerName']['conv']['ParallelizationFactor'] = 12

hls_config['LayerName']['relu1']['Precision']['result'] = 'ap_ufixed<10, 4>'

hls_config['LayerName']['dense1']['Precision']['accum'] = 'ap_fixed<22, 8>'
hls_config['LayerName']['dense1']['Precision']['result'] = 'ap_fixed<14, 8>'
hls_config['LayerName']['dense1_linear']['Precision'] = 'ap_fixed<14, 8>'

hls_config['LayerName']['relu2']['Precision']['result'] = 'ap_ufixed<10, 4>' #16 ok

hls_config['LayerName']['output']['Precision']['accum'] = 'ap_fixed<16, 8>'
hls_config['LayerName']['output']['Precision']['result'] = 'ap_ufixed<16, 8>'
hls_config['LayerName']['output_linear']['Precision'] = 'ap_ufixed<16, 8>'

cfg = hls4ml.converters.create_config(part="xc7vx690tffg1927-2")

cfg['IOType'] = 'io_parallel'
cfg['HLSConfig'] = hls_config
cfg['KerasModel'] = model
cfg['ClockPeriod']  = 6.25
cfg['OutputDir']  = 'cicada/'
cfg['Part'] = 'xc7vx690tffg1927-2'

In [ ]:
hls_model = hls4ml.converters.keras_to_hls(cfg)

hls4ml.model.optimizer.get_optimizer(
    'output_rounding_saturation_mode'
).configure(layers=[])
hls_model.compile()

In [ ]:
hls4ml.utils.plot_model(hls_model, show_shapes=False, show_precision=True, to_file=None)

In [ ]:
hls4ml.model.profiling.numerical(model=model, hls_model=hls_model);

# Testing

In [ ]:
ACCEPTED_ERROR = 1.4

In [ ]:
# Test vector, zeros:
tv_0 = np.zeros((1, 252)) + 0.

# Test vector, ones:
tv_1 = np.zeros((1, 252)) + 1.

# Test vector, mean zero bias, 2018 run D:
tv_zb = np.array([[2, 1, 1, 2, 1, 2, 3, 3, 3, 2, 3, 1, 1, 2],
       [2, 1, 1, 2, 2, 2, 3, 3, 2, 2, 3, 1, 1, 2],
       [2, 1, 1, 1, 2, 2, 3, 3, 2, 2, 3, 1, 1, 2],
       [2, 1, 1, 2, 1, 2, 3, 3, 2, 1, 2, 1, 1, 2],
       [2, 1, 1, 2, 2, 2, 3, 4, 3, 2, 4, 1, 1, 2],
       [2, 1, 1, 2, 2, 2, 3, 3, 2, 2, 2, 1, 1, 2],
       [2, 1, 1, 3, 2, 3, 5, 2, 2, 1, 3, 1, 1, 2],
       [2, 1, 1, 2, 2, 3, 4, 3, 2, 2, 2, 1, 1, 3],
       [2, 1, 1, 2, 2, 3, 4, 3, 2, 1, 2, 1, 1, 3],
       [2, 1, 1, 2, 2, 2, 3, 3, 2, 1, 2, 1, 1, 2],
       [2, 1, 1, 2, 2, 2, 3, 3, 2, 1, 2, 1, 1, 2],
       [2, 1, 1, 2, 2, 2, 3, 3, 2, 1, 2, 1, 1, 2],
       [2, 1, 1, 2, 2, 2, 3, 3, 2, 1, 3, 1, 1, 2],
       [2, 1, 1, 2, 2, 2, 3, 3, 2, 1, 2, 1, 1, 2],
       [0, 0, 0, 2, 2, 2, 3, 3, 2, 2, 2, 1, 1, 2],
       [0, 0, 0, 3, 2, 3, 5, 4, 3, 2, 4, 1, 1, 2],
       [2, 1, 1, 2, 2, 2, 4, 3, 2, 2, 4, 1, 1, 2],
       [2, 1, 1, 2, 2, 2, 3, 3, 2, 1, 3, 1, 1, 2]]).reshape(1, 252) + 0.

# Test vector, signal haa4b_ma15_powheg average:
tv_sig = np.array([[8, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 2, 2, 8],
       [9, 2, 2, 3, 3, 3, 3, 3, 3, 2, 3, 2, 2, 9],
       [9, 2, 2, 3, 2, 3, 3, 3, 3, 3, 3, 2, 2, 9],
       [9, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 2, 2, 9],
       [8, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 2, 2, 8],
       [8, 2, 2, 3, 2, 3, 3, 3, 3, 3, 3, 2, 2, 8],
       [9, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 2, 2, 9],
       [9, 2, 2, 3, 3, 3, 3, 3, 3, 2, 3, 2, 2, 9],
       [9, 2, 2, 3, 3, 3, 3, 3, 3, 2, 3, 2, 2, 8],
       [8, 2, 2, 3, 3, 3, 3, 3, 3, 2, 3, 2, 2, 8],
       [9, 2, 2, 3, 3, 3, 3, 3, 3, 2, 3, 2, 2, 8],
       [9, 2, 2, 3, 2, 3, 3, 3, 3, 3, 3, 2, 2, 9],
       [9, 2, 2, 3, 2, 3, 3, 3, 3, 2, 3, 2, 2, 9],
       [8, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 2, 2, 8],
       [8, 2, 2, 3, 3, 3, 3, 3, 3, 2, 3, 2, 2, 8],
       [9, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 2, 2, 9],
       [9, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 2, 2, 9],
       [8, 2, 2, 3, 2, 3, 3, 3, 3, 2, 3, 2, 2, 8]]).reshape(1, 252) + 0.

# Test vector, hand made:
tv_hand = np.array([[2, 1, 1, 2, 1, 2, 3, 3, 3, 2, 3, 1, 1, 2],
       [2, 1, 1, 2, 2, 2, 3, 3, 2, 2, 3, 1, 1, 2],
       [2, 1, 1, 1, 2, 2, 1023, 3, 2, 2, 3, 1, 1, 2],
       [2, 1, 1, 2, 1, 2, 3, 3, 2, 1, 2, 1, 1, 2],
       [2, 1, 1, 1023, 1023, 2, 3, 4, 3, 2, 4, 1023, 1, 2],
       [2, 1, 1023, 1023, 2, 2, 3, 3, 2, 2, 2, 1, 1, 2],
       [2, 1, 1, 3, 2, 3, 5, 2, 2, 1, 3, 1023, 1, 2],
       [2, 1, 1, 2, 2, 3, 4, 3, 2, 2, 2, 1, 1, 3],
       [2, 1, 1, 2, 2, 3, 4, 3, 2, 1, 2, 1, 1, 3],
       [2, 1, 1, 2, 2, 2, 3, 3, 1023, 1, 2, 1, 1, 2],
       [2, 1, 1, 1023, 2, 2, 3, 3, 2, 1, 2, 1, 1, 2],
       [2, 1, 1, 2, 2, 2, 3, 3, 2, 1, 2, 1, 1, 2],
       [2, 1, 1, 2, 2, 2, 3, 3, 2, 1, 3, 1, 1, 2],
       [2, 1, 1, 2, 1023, 2, 3, 3, 2, 1, 2, 1, 1, 2],
       [0, 1023, 0, 2, 2, 2, 3, 3, 2, 2, 2, 1, 1, 2],
       [0, 0, 0, 3, 2, 3, 5, 4, 3, 2, 4, 1, 1, 2],
       [2, 1, 1, 2, 2, 2, 4, 3, 2, 2, 1023, 1, 1, 2],
       [2, 1023, 1, 2, 2, 2, 3, 3, 2, 1, 3, 1, 1, 2]]).reshape(1, 252) + 0.

In [ ]:
for tv in [tv_0, tv_1, tv_zb, tv_sig, tv_hand]:
    print(abs(model.predict(tv) - hls_model.predict(tv)) <= ACCEPTED_ERROR)

In [ ]:
# Real signal samples

scores = []
errors = []

for dataset in [
    '/eos/user/a/adpol/L1AD/Signal/120X/haa4taus_ma15_powheg.h5',
    '/eos/user/a/adpol/L1AD/Signal/120X/haa4b_ma50_powheg.h5',
    '/eos/user/a/adpol/L1AD/Signal/120X/emj-mMed-800-mDark-10-ctau-1000.h5']:
    print(dataset)
    X_test = h5py.File(dataset,'r')['CaloRegions']
    for idx in range(min(4000, len(X_test))):
        vector = X_test[idx].reshape(1, 252) + 0.
        score_keras = float(model.predict(vector))
        score_hls4ml = hls_model.predict(vector)
        diff = abs(score_keras - score_hls4ml)
        if diff > ACCEPTED_ERROR:
            print('Error', diff)
        errors.append(diff)
        scores.append(score_keras)

In [ ]:
import matplotlib
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 10))
plt.scatter(scores, errors, s=1)
plt.xlabel('Anomaly Score, $S$', fontsize=18)
plt.ylabel('Error, $|S_{Keras} - S_{hls4ml}|$', fontsize=18)

# QKeras and HLS4ML Comparison

In [ ]:
ZeroBias = h5py.File('/eos/user/a/adpol/L1AD/Background/EphemeralZeroBias2018RunD_1.h5', 'r')
ZeroBias = ZeroBias['CaloRegions'][:500000].astype('float32').reshape(-1, 252)
print('ZeroBias shape: ' + str(ZeroBias.shape))

MC_files = ['/eos/user/a/adpol/L1AD/Signal/120X/GluGluHToTauTau_M-125_TuneCP5_14TeV.h5',
            '/eos/user/a/adpol/L1AD/Signal/120X/GluGluToHHTo4B_node_cHHH1_TuneCP5_14TeV.h5',
            '/eos/user/a/adpol/L1AD/Signal/120X/TT_TuneCP5_14TeV.h5',
            '/eos/user/a/adpol/L1AD/Signal/120X/haa4b_ma15_powheg.h5']
MC = []
for i in range(len(MC_files)):
    MC.append(h5py.File(MC_files[i], 'r'))
    MC[i] = MC[i]['CaloRegions'][:500000].astype('float32').reshape(-1, 252)
    print('i = ' + str(i) + ': ' + str(MC[i].shape))

In [ ]:
from sklearn.model_selection import train_test_split

_, X_test = train_test_split(ZeroBias, test_size=0.4, random_state=123)

In [ ]:
X_test_predict_keras = model.predict(X_test)
X_test_predict_hls = hls_model.predict(X_test)

In [ ]:
MC_predict_keras = []
for i in range(len(MC)):
    MC_predict_keras.append(model.predict(MC[i]))

MC_predict_hls = []
for i in range(len(MC)):
    MC_predict_hls.append(hls_model.predict(MC[i]))

In [ ]:
Y_zb = np.zeros((X_test.shape[0], 1))
Y_mc = []
for i in range(len(MC)):
    Y_mc.append(np.ones((MC[i].shape[0], 1)))

Y_true = []
Y_model_hls = []
Y_model_keras = []

for i in range(len(MC)):
    Y_true.append(np.concatenate((Y_mc[i], Y_zb)))
    Y_model_keras.append(np.concatenate((MC_predict_keras[i], X_test_predict_keras)))
    Y_model_hls.append(np.concatenate((MC_predict_hls[i], X_test_predict_hls)))

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc

plt.figure(figsize = (10, 10))
axes = plt.subplot(1, 1, 1)
fpr_model = []
tpr_model = []
thresholds_model = []
roc_auc_model = []
cmap = ['green', 'red', 'blue', 'orange']
labels = ['H->tautau', 'SM HH->4b', 'TTbar', 'H->aa->4b']
for y_true, y_score_keras, y_score_hls, c, label in zip(Y_true, Y_model_keras, Y_model_hls, cmap, labels):
    
    fpr, tpr, thresholds = roc_curve(y_true, y_score_keras)
    roc_auc = auc(fpr, tpr)
    fpr *= 28.61
    
    axes.plot(fpr,
              tpr,
              linestyle = '-',
              lw = 1.5,
              color=c,
              label = 'Keras: {0}, AUC = {1:.5f}'.format(label, roc_auc))
    
    fpr, tpr, thresholds = roc_curve(y_true, y_score_hls)
    roc_auc = auc(fpr, tpr)
    fpr *= 28.61
    
    axes.plot(fpr,
              tpr,
              linestyle = '--',
              lw = 1.5,
              color=c,
              label = 'hls4ml: {0}, AUC = {1:.5f}'.format(label, roc_auc))

axes.plot([0.005, 0.005], [0, 1], linestyle = '--', lw = 1.5, color = 'black', label = '5 kHz trigger rate')
axes.set_xlim([0.0002861, 28.61])
axes.set_ylim([0.001, 1.0])
axes.set_xscale(value = "log")
axes.set_yscale(value = "log")
axes.set_xlabel('Trigger Rate (MHz)', fontsize=18)
axes.set_ylabel('Signal Efficiency', fontsize=18)
axes.set_title('QKeras vs. hls4ml Comparison', fontsize=18)
axes.legend(fontsize=18)
plt.show()

# Debug

In [ ]:
model.predict(tv_0), hls_model.predict(tv_0)

In [ ]:
hls4ml_pred, hls4ml_trace = hls_model.trace(tv_0)
keras_trace = hls4ml.model.profiling.get_ymodel_keras(model, tv_0)

# Synthesis

In [ ]:
# Change the uncertainty in tcl: set_clock_uncertainty 30% {get_clocks default}

!grep -n create_clock cicada/build_prj.tcl | awk -F ":" '{print$1}' | ( read line; echo "$((line + 1)) ") | xargs -I {} sed -i '{} i set_clock_uncertainty 30% {get_clocks default}' cicada/build_prj.tcl 


In [ ]:
# Run Vivado HLS

!vivado_hls -f cicada/build_prj.tcl "reset=1 synth=1 csim=0 cosim=0 validation=0 export=0 vsynth=0"


In [ ]:
# Get the report

!cat cicada/myproject_prj/solution1/syn/report/myproject_csynth.rpt
